In [1]:
import numpy as np
import statsmodels.api as sm

# ----------------------------
# Configuración de la simulación
# ----------------------------

T = 200          # longitud de cada serie
H = 12           # últimos 12 períodos para evaluación
N_SIM = 500      # número de simulaciones Monte Carlo

phi_true = 0.6   # parámetro AR(1) verdadero
c_true = 0.0     # constante verdadera
sigma_eps = 1.0  # desviación estándar del ruido

np.random.seed(123)  # para reproducibilidad

# Para guardar resultados
all_errors = np.zeros((N_SIM, H))   # errores de pronóstico para cada simulación y horizonte
phi_hats = np.zeros(N_SIM)          # estimaciones de phi en cada simulación

# ----------------------------
# Función para simular un AR(1)
# ----------------------------

def simulate_ar1(T, phi, c, sigma):
    """
    Simula un proceso AR(1):
        y_t = c + phi * y_{t-1} + eps_t
    con eps_t ~ N(0, sigma^2)
    """
    eps = np.random.normal(loc=0.0, scale=sigma, size=T)
    y = np.zeros(T)
    # y_0 = 0
    for t in range(1, T):
        y[t] = c + phi * y[t - 1] + eps[t]
    return y

# ----------------------------
# Bucle Monte Carlo
# ----------------------------

for i in range(N_SIM):
    # 1) Generar serie AR(1)
    y = simulate_ar1(T=T, phi=phi_true, c=c_true, sigma=sigma_eps)
    
    # 2) Separar entrenamiento y test (últimos 12 puntos)
    y_train = y[:-H]  # 0 ... T-H-1
    y_test  = y[-H:]  # últimos H puntos reales
    
    # 3) Estimar un modelo AR(1) sobre y_train
    #    Usamos AutoReg con un rezago
    model = sm.tsa.AutoReg(y_train, lags=1, old_names=False)
    res = model.fit()
    
    # Guardar la estimación de phi (coeficiente del rezago 1)
    # params[0] = const, params[1] = phi en AR(1)
    phi_hat = res.params[1]
    phi_hats[i] = phi_hat
    
    # 4) Pronosticar los últimos 12 períodos (out-of-sample)
    # start = len(y_train) -> primer índice después del train
    # end   = len(y_train) + H - 1 -> último índice pronosticado
    y_pred = res.predict(start=len(y_train), end=len(y_train) + H - 1, dynamic=False)
    
    # 5) Calcular errores de pronóstico (real - estimado)
    errors = y_test - y_pred
    all_errors[i, :] = errors

# ----------------------------
# Análisis de resultados
# ----------------------------

# Error medio global (promedio sobre simulaciones y horizontes)
mean_error_global = all_errors.mean()

# Error medio por horizonte (1 a 12 pasos)
mean_error_by_h = all_errors.mean(axis=0)

# Sesgo del estimador de phi: E[phi_hat - phi_true]
bias_phi = phi_hats.mean() - phi_true

print("----------------------------------------------------")
print("Resultados Monte Carlo ({} simulaciones)".format(N_SIM))
print("DGP: AR(1) con phi = {:.2f}, c = {:.2f}, sigma = {:.2f}".format(phi_true, c_true, sigma_eps))
print("Serie de longitud T = {}, últimos H = {} períodos para test\n".format(T, H))

print("Error medio de pronóstico (global, todos los horizontes):")
print("  E[y_real - y_pred] ≈ {:.6f}".format(mean_error_global))

print("\nError medio de pronóstico por horizonte (1-step, 2-step, ..., 12-step):")
for h in range(H):
    print("  Horizonte {}: {:.6f}".format(h + 1, mean_error_by_h[h]))

print("\nSesgo en la estimación de phi (E[phi_hat] - phi_true):")
print("  Bias(phi_hat) ≈ {:.6f}".format(bias_phi))
print("  E[phi_hat] ≈ {:.6f} (phi_true = {:.2f})".format(phi_hats.mean(), phi_true))
print("----------------------------------------------------")

----------------------------------------------------
Resultados Monte Carlo (500 simulaciones)
DGP: AR(1) con phi = 0.60, c = 0.00, sigma = 1.00
Serie de longitud T = 200, últimos H = 12 períodos para test

Error medio de pronóstico (global, todos los horizontes):
  E[y_real - y_pred] ≈ -0.008579

Error medio de pronóstico por horizonte (1-step, 2-step, ..., 12-step):
  Horizonte 1: 0.023707
  Horizonte 2: 0.028731
  Horizonte 3: 0.035072
  Horizonte 4: 0.051231
  Horizonte 5: 0.010460
  Horizonte 6: -0.083047
  Horizonte 7: -0.031268
  Horizonte 8: -0.053490
  Horizonte 9: -0.037699
  Horizonte 10: -0.006326
  Horizonte 11: 0.007291
  Horizonte 12: -0.047611

Sesgo en la estimación de phi (E[phi_hat] - phi_true):
  Bias(phi_hat) ≈ -0.016977
  E[phi_hat] ≈ 0.583023 (phi_true = 0.60)
----------------------------------------------------
